In [ ]:
# Import SPACEc and required libraries
import spacec as sp
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
from PIL import Image
import os
import glob
import warnings
from pathlib import Path
import pickle
from sklearn.metrics import classification_report

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Set up plotting parameters
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 300
plt.rc('axes', grid=False)
sc.settings.set_figure_params(dpi=80, facecolor='white')

# Set default colormap
plt.rcParams["image.cmap"] = 'viridis'

print("✓ Libraries imported successfully")
print("✓ Plotting parameters configured")

# Check for GPU availability (optional for faster segmentation)
try:
    sp.hf.check_for_gpu()
except:
    print("GPU not available, will use CPU for segmentation")


In [ ]:
# Define data paths
root_path = Path.cwd().parent.parent  # hexif root directory
data_path = root_path / 'data' / 'nomad_data' / 'CODEX'
output_dir = root_path / 'output' / 'spacec_analysis'

# Create output directory if it doesn't exist
output_dir.mkdir(parents=True, exist_ok=True)

# Print paths for verification
print(f"Root path: {root_path}")
print(f"Data path: {data_path}")
print(f"Output directory: {output_dir}")
print(f"Data path exists: {data_path.exists()}")

# Get list of TMA directories
tma_dirs = [d for d in data_path.iterdir() if d.is_dir() and 'TMA' in d.name]
print(f"\nFound {len(tma_dirs)} TMA directories:")
for tma_dir in sorted(tma_dirs):
    print(f"  - {tma_dir.name}")

# Load marker list
marker_file = data_path / 'MarkerList.txt'
if marker_file.exists():
    with open(marker_file, 'r') as f:
        markers = [line.strip() for line in f.readlines()]
    print(f"\nLoaded {len(markers)} markers from MarkerList.txt")
    print(f"Markers: {', '.join(markers[:10])}{'...' if len(markers) > 10 else ''}")
else:
    print("MarkerList.txt not found!")

# Load TMA mapping files
mapping_files = list(data_path.glob('*TMA*map*.xlsx'))
print(f"\nFound {len(mapping_files)} TMA mapping files:")
for mapping_file in mapping_files:
    print(f"  - {mapping_file.name}")


In [ ]:
def inspect_tma_structure(tma_dir):
    """Inspect the structure of a TMA directory"""
    print(f"\n{'='*50}")
    print(f"Inspecting: {tma_dir.name}")
    print(f"{'='*50}")
    
    # Check for bestFocus directory
    bestfocus_dir = tma_dir / 'bestFocus'
    if bestfocus_dir.exists():
        tif_files = list(bestfocus_dir.glob('*.tif'))
        print(f"bestFocus directory: {len(tif_files)} TIF files")
        if tif_files:
            print(f"  Sample files: {', '.join([f.name for f in tif_files[:3]])}...")
    
    # Check for Scan1 directory
    scan1_dir = tma_dir / 'Scan1'
    if scan1_dir.exists():
        qptiff_files = list(scan1_dir.glob('*.qptiff'))
        xpd_files = list(scan1_dir.glob('*.xpd'))
        marker_files = list(scan1_dir.glob('MarkerList.txt'))
        
        print(f"Scan1 directory contents:")
        print(f"  - QPTIFF files: {len(qptiff_files)}")
        print(f"  - XPD files: {len(xpd_files)}")
        print(f"  - Marker files: {len(marker_files)}")
        
        # Check .temp directory
        temp_dir = scan1_dir / '.temp'
        if temp_dir.exists():
            temp_files = list(temp_dir.glob('*'))
            print(f"  - .temp directory: {len(temp_files)} files")
    
    return bestfocus_dir, scan1_dir

# Inspect all TMA directories
tma_info = {}
for tma_dir in sorted(tma_dirs):
    bestfocus_dir, scan1_dir = inspect_tma_structure(tma_dir)
    tma_info[tma_dir.name] = {
        'path': tma_dir,
        'bestfocus': bestfocus_dir,
        'scan1': scan1_dir
    }


In [ ]:
def create_thumbnail_grid(tif_files, max_images=12, figsize=(15, 10)):
    """Create a grid of thumbnail images from TIF files"""
    
    # Limit the number of images
    tif_files = tif_files[:max_images]
    
    # Calculate grid dimensions
    n_images = len(tif_files)
    n_cols = min(4, n_images)
    n_rows = (n_images + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=figsize)
    
    # Ensure axes is always a 2D array
    if n_rows == 1 and n_cols == 1:
        axes = [[axes]]
    elif n_rows == 1:
        axes = [axes]
    elif n_cols == 1:
        axes = [[ax] for ax in axes]
    
    for i, tif_file in enumerate(tif_files):
        row = i // n_cols
        col = i % n_cols
        ax = axes[row][col]
        
        try:
            # Try to load TIF file - use PIL as fallback
            try:
                import tifffile
                img = tifffile.imread(str(tif_file))
            except:
                from PIL import Image
                img = np.array(Image.open(str(tif_file)))
            
            # Handle different array dimensions
            if img.ndim == 3:
                # Use first channel, normalize for display
                img_display = img[0] if img.shape[0] < img.shape[2] else img[:, :, 0]
            else:
                img_display = img
                
            # Normalize for display
            if img_display.max() > img_display.min():
                img_display = (img_display - img_display.min()) / (img_display.max() - img_display.min())
            
            ax.imshow(img_display, cmap='gray')
            ax.set_title(f"{tif_file.name}", fontsize=8)
            ax.axis('off')
            
        except Exception as e:
            ax.text(0.5, 0.5, f"Error loading\\n{tif_file.name}", 
                   ha='center', va='center', transform=ax.transAxes, fontsize=8)
            ax.set_title(f"{tif_file.name}", fontsize=8)
            ax.axis('off')
    
    # Hide unused subplots
    for i in range(n_images, n_rows * n_cols):
        row = i // n_cols
        col = i % n_cols
        ax = axes[row][col]
        ax.axis('off')
    
    plt.tight_layout()
    return fig

# Create thumbnail grids for each TMA
print("\\nCreating thumbnail visualizations...")
for tma_name, info in tma_info.items():
    if info['bestfocus'].exists():
        tif_files = list(info['bestfocus'].glob('*.tif'))
        if tif_files:
            print(f"\\nThumbnails for {tma_name}:")
            fig = create_thumbnail_grid(tif_files, max_images=12)
            plt.suptitle(f"{tma_name} - bestFocus TIF Files", fontsize=14, y=0.98)
            plt.show()
            
            # Save thumbnail grid
            thumbnail_path = output_dir / f"{tma_name}_thumbnails.png"
            fig.savefig(thumbnail_path, dpi=300, bbox_inches='tight')
            print(f"Saved thumbnail grid to: {thumbnail_path}")
            plt.close()
    else:
        print(f"No bestFocus directory found for {tma_name}")


In [ ]:
# Select a representative TMA for detailed analysis
# For this example, we'll use the first ccRCC TMA
selected_tma = None
for tma_name, info in tma_info.items():
    if 'ccRCC' in tma_name and info['bestfocus'].exists():
        selected_tma = info
        selected_tma_name = tma_name
        break

if selected_tma is None:
    print("No suitable TMA found for analysis")
else:
    print(f"Selected TMA for analysis: {selected_tma_name}")
    
    # Get list of TIF files in bestFocus directory
    tif_files = list(selected_tma['bestfocus'].glob('*.tif'))
    print(f"Found {len(tif_files)} TIF files")
    
    # Create a channel names file for the selected TMA
    channel_file = output_dir / f"{selected_tma_name}_channelnames.txt"
    with open(channel_file, 'w') as f:
        for marker in markers:
            f.write(f"{marker}\\n")
    
    print(f"Created channel names file: {channel_file}")
    
    # Select first few TIF files for segmentation (to manage processing time)
    selected_tifs = tif_files[:3]  # Process first 3 TIF files
    print(f"Selected {len(selected_tifs)} TIF files for segmentation:")
    for tif in selected_tifs:
        print(f"  - {tif.name}")


In [ ]:
# Visualize segmentation channels before segmentation
if selected_tma and len(selected_tifs) > 0:
    sample_tif = selected_tifs[0]
    print(f"\\nVisualizing segmentation channels for {sample_tif.name}...")
    
    # Define membrane markers for segmentation
    membrane_markers = ['CD45', 'CD31', 'panCK']  # Adjust based on your markers
    available_membrane = [m for m in membrane_markers if m in markers]
    
    if available_membrane:
        print(f"Available membrane markers: {available_membrane}")
        
        # Visualize segmentation channels
        sp.pl.segmentation_ch(
            file_name=str(sample_tif),
            channel_file=str(channel_file),
            output_dir=str(output_dir),
            extra_seg_ch_list=available_membrane,
            nuclei_channel='DAPI',
            input_format='CODEX'
        )
    else:
        print("No suitable membrane markers found, using nuclei only")
        
        # Visualize nuclei channel only
        sp.pl.segmentation_ch(
            file_name=str(sample_tif),
            channel_file=str(channel_file),
            output_dir=str(output_dir),
            nuclei_channel='DAPI',
            input_format='CODEX'
        )
        
print("\\nSegmentation channel visualization complete.")


In [ ]:
# Perform cell segmentation using Mesmer
if selected_tma and len(selected_tifs) > 0:
    print("\\nStarting cell segmentation...")
    
    # Store segmentation results
    segmentation_results = {}
    
    for i, tif_file in enumerate(selected_tifs):
        print(f"\\nProcessing {tif_file.name} ({i+1}/{len(selected_tifs)})...")
        
        # Define output filename
        output_name = f"{selected_tma_name}_{tif_file.stem}"
        
        # Perform segmentation
        try:
            seg_output = sp.tl.cell_segmentation(
                file_name=str(tif_file),
                channel_file=str(channel_file),
                output_dir=str(output_dir),
                seg_method='mesmer',  # Use Mesmer for segmentation
                nuclei_channel='DAPI',
                output_fname=output_name,
                membrane_channel_list=available_membrane if available_membrane else None,
                compartment='whole-cell',  # Segment whole cells
                input_format='CODEX',
                resize_factor=1,  # Adjust if images are too large
                size_cutoff=0  # Minimum cell size filter
            )
            
            # Store results
            segmentation_results[tif_file.name] = {
                'seg_output': seg_output,
                'output_name': output_name
            }
            
            print(f"✓ Segmentation completed for {tif_file.name}")
            
        except Exception as e:
            print(f"✗ Error processing {tif_file.name}: {str(e)}")
            continue
    
    print(f"\\nSegmentation completed for {len(segmentation_results)} files")
else:
    print("No files selected for segmentation")


In [ ]:
# Visualize segmentation results
if segmentation_results:
    print("\\nVisualizing segmentation results...")
    
    # Create overlay data for visualization
    overlay_data = {}
    
    for tif_name, seg_info in segmentation_results.items():
        print(f"\\nCreating overlay for {tif_name}...")
        
        try:
            # Create overlay visualization
            overlay_data_single, rgb_images = sp.pl.show_masks(
                seg_output=seg_info['seg_output'],
                nucleus_channel='DAPI',
                additional_channels=available_membrane if available_membrane else ['CD45'],
                show_subsample=True,
                n=4,  # Number of subsamples
                tilesize=400,  # Size of tiles
                rand_seed=42
            )
            
            # Store overlay data
            overlay_data[tif_name] = overlay_data_single
            
            print(f"✓ Overlay created for {tif_name}")
            
        except Exception as e:
            print(f"✗ Error creating overlay for {tif_name}: {str(e)}")
            continue
    
    # Save segmentation results
    print("\\nSaving segmentation results...")
    for tif_name, seg_info in segmentation_results.items():
        try:
            # Save segmentation output
            seg_pickle_path = output_dir / f"{seg_info['output_name']}_segmentation.pickle"
            with open(seg_pickle_path, 'wb') as f:
                pickle.dump(seg_info['seg_output'], f)
            
            # Save overlay data if available
            if tif_name in overlay_data:
                overlay_pickle_path = output_dir / f"{seg_info['output_name']}_overlay.pickle"
                with open(overlay_pickle_path, 'wb') as f:
                    pickle.dump(overlay_data[tif_name], f)
            
            print(f"✓ Saved results for {tif_name}")
            
        except Exception as e:
            print(f"✗ Error saving results for {tif_name}: {str(e)}")
            continue
    
    print("\\nSegmentation results saved successfully!")
else:
    print("No segmentation results to visualize")


In [ ]:
# Read segmentation results and create dataframe
if segmentation_results:
    print("Reading segmentation results...")
    
    # Collect CSV files from segmentation
    csv_files = []
    region_names = []
    meta_info = []
    
    for tif_name, seg_info in segmentation_results.items():
        csv_file = output_dir / f"{seg_info['output_name']}_mesmer_result.csv"
        if csv_file.exists():
            csv_files.append(str(csv_file))
            region_names.append(tif_name.replace('.tif', ''))
            
            # Extract metadata from TIF filename
            if 'ccRCC' in tif_name:
                meta_info.append('ccRCC')
            elif 'ccOC' in tif_name:
                meta_info.append('ccOC')
            else:
                meta_info.append('unknown')
    
    print(f"Found {len(csv_files)} CSV files for processing")
    
    # Read and concatenate segmentation results
    if csv_files:
        df_seg = sp.pp.read_segdf(
            segfile_list=csv_files,
            seg_method='mesmer',
            region_list=region_names,
            meta_list=meta_info
        )
        
        print(f"Combined dataframe shape: {df_seg.shape}")
        print(f"Columns: {list(df_seg.columns)}")
        
        # Display first few rows
        print("\\nFirst 5 rows of segmentation data:")
        print(df_seg.head())
        
    else:
        print("No CSV files found from segmentation")
        df_seg = None
else:
    print("No segmentation results available")
    df_seg = None


In [ ]:
# Filter cells by DAPI intensity and area
if df_seg is not None:
    print("\\nFiltering cells by size and nuclear intensity...")
    
    # Calculate filtering thresholds
    one_percent_area = np.percentile(df_seg.area, 1)
    one_percent_nuc = np.percentile(df_seg.DAPI, 1)
    
    print(f"1% area threshold: {one_percent_area:.2f}")
    print(f"1% DAPI threshold: {one_percent_nuc:.2f}")
    
    # Apply filters
    df_filt = sp.pp.filter_data(
        df_seg,
        nuc_thres=one_percent_nuc,
        size_thres=one_percent_area,
        nuc_marker="DAPI",
        cell_size="area",
        region_column="region_num",
        color_by="region_num",
        log_scale=False
    )
    
    print(f"Filtered dataframe shape: {df_filt.shape}")
    print(f"Removed {len(df_seg) - len(df_filt)} cells ({((len(df_seg) - len(df_filt))/len(df_seg)*100):.1f}%)")
    
    # Show distribution of cells by region
    print("\\nCells per region after filtering:")
    print(df_filt.groupby('unique_region').size())
    
else:
    print("No segmentation data available for filtering")
    df_filt = None


In [ ]:
# Normalize data per region
if df_filt is not None:
    print("\\nNormalizing protein expression data...")
    
    # Identify columns to exclude from normalization
    exclude_cols = ['eccentricity', 'perimeter', 'convex_area', 'axis_major_length', 
                   'axis_minor_length', 'label']
    keep_cols = ['DAPI', 'x', 'y', 'area', 'region_num', 'unique_region', 'condition']
    
    # Normalize data per region
    dfz = pd.DataFrame()
    
    for region in df_filt.unique_region.unique():
        df_reg = df_filt[df_filt.unique_region == region]
        
        # Normalize using z-score
        df_reg_norm = sp.pp.format(
            data=df_reg,
            list_out=exclude_cols,
            list_keep=keep_cols,
            method="zscore"
        )
        
        dfz = pd.concat([dfz, df_reg_norm], axis=0)
    
    print(f"Normalized dataframe shape: {dfz.shape}")
    
    # Find the column index for the last marker
    protein_cols = [col for col in dfz.columns if col not in keep_cols]
    if protein_cols:
        last_marker_idx = dfz.columns.get_loc(protein_cols[-1])
        print(f"Last marker column index: {last_marker_idx}")
        print(f"Protein markers: {protein_cols}")
    else:
        print("No protein markers found")
        last_marker_idx = None
        
else:
    print("No filtered data available for normalization")
    dfz = None
    last_marker_idx = None


In [ ]:
# Remove noisy cells
if dfz is not None and last_marker_idx is not None:
    print("\\nRemoving noisy cells...")
    
    # Determine noise thresholds per region
    cutoff_dict = {}
    for region in dfz.unique_region.unique():
        df_reg = dfz[dfz.unique_region == region]
        
        # Visualize z-count threshold
        print(f"\\nAnalyzing noise threshold for region: {region}")
        sp.pl.zcount_thres(
            dfz=df_reg,
            col_num=last_marker_idx,
            cut_off=0.01,  # Top 1% of cells
            count_bin=50
        )
        
        # Calculate thresholds (you might need to adjust these based on the plots)
        # For demonstration, we'll use automated thresholds
        end_idx = last_marker_idx + 1
        protein_data = df_reg.iloc[:, :end_idx]
        z_counts = (protein_data > 0).sum(axis=1)
        z_sums = protein_data.sum(axis=1)
        
        z_count_threshold = np.percentile(z_counts, 99)  # Top 1%
        z_sum_threshold = np.percentile(z_sums, 99)  # Top 1%
        
        cutoff_dict[region] = [z_count_threshold, z_sum_threshold]
        print(f"  Z-count threshold: {z_count_threshold:.1f}")
        print(f"  Z-sum threshold: {z_sum_threshold:.1f}")
    
    # Apply noise removal
    df_nn = pd.DataFrame()
    
    for region in dfz.unique_region.unique():
        df_reg = dfz[dfz.unique_region == region]
        
        df_reg_nn, removed_cells = sp.pp.remove_noise(
            df=df_reg,
            col_num=last_marker_idx,
            z_count_thres=cutoff_dict[region][0],
            z_sum_thres=cutoff_dict[region][1]
        )
        
        print(f"Region {region}: Removed {removed_cells} noisy cells")
        print(f"  Remaining cells: {len(df_reg_nn)}")
        
        df_nn = pd.concat([df_nn, df_reg_nn], axis=0)
    
    print(f"\\nFinal dataframe shape after noise removal: {df_nn.shape}")
    print(f"Total cells removed: {len(dfz) - len(df_nn)}")
    
    # Save the preprocessed data
    df_nn.to_csv(output_dir / "preprocessed_data.csv", index=False)
    print(f"Preprocessed data saved to: {output_dir / 'preprocessed_data.csv'}")
    
else:
    print("No normalized data available for noise removal")
    df_nn = None


In [ ]:
# Create AnnData object for clustering
if df_nn is not None:
    print("Creating AnnData object for clustering...")
    
    # Create AnnData object compatible with scanpy
    adata = sp.hf.make_anndata(
        df_nn=df_nn,
        col_sum=last_marker_idx,
        nonFuncAb_list=[]  # Remove non-functional antibodies if needed
    )
    
    print(f"AnnData object created: {adata}")
    print(f"Number of cells: {adata.n_obs}")
    print(f"Number of features: {adata.n_vars}")
    
    # Save the AnnData object
    adata.write_h5ad(output_dir / 'preprocessed_adata.h5ad')
    print(f"AnnData saved to: {output_dir / 'preprocessed_adata.h5ad'}")
    
    # Display basic information
    print("\\nMarkers for clustering:")
    print(list(adata.var_names)[:10], "..." if len(adata.var_names) > 10 else "")
    print(f"\\nConditions: {adata.obs['condition'].unique()}")
    print(f"Regions: {adata.obs['unique_region'].unique()}")
    
else:
    print("No preprocessed data available for clustering")
    adata = None


In [ ]:
# Perform clustering
if adata is not None:
    print("\\nPerforming clustering...")
    
    # Set random seed for reproducibility
    clustering_random_seed = 42
    
    # Define markers for clustering (cancer-relevant markers)
    clustering_markers = [
        'CD3', 'CD4', 'CD8', 'CD20', 'CD68', 'CD45', 'panCK', 'Vimentin',
        'CD31', 'CD34', 'FoxP3', 'Ki67', 'PD1', 'PDL1', 'CD11c', 'CD206',
        'CD57', 'CD56', 'CD138', 'CD25', 'GZMB', 'HLA-DR', 'CD15', 'CD163',
        'CD11b', 'CD90', 'aSMA', 'Ecadherin', 'CA9', 'HE4', 'VEGFR1', 'VEGFR2'
    ]
    
    # Filter markers that are present in the data
    available_markers = [m for m in clustering_markers if m in adata.var_names]
    print(f"Available markers for clustering: {len(available_markers)}")
    print(f"Markers: {available_markers}")
    
    # Perform clustering
    adata = sp.tl.clustering(
        adata,
        clustering='leiden',
        n_neighbors=15,
        resolution=1.0,
        reclustering=False,
        marker_list=available_markers,
        seed=clustering_random_seed
    )
    
    # Visualize clustering results
    print("\\nVisualizing clustering results...")
    
    # UMAP visualization
    sc.pl.umap(adata, color=['leiden_1', 'unique_region'], wspace=0.5, 
               save='_clustering_overview.pdf')
    
    # Create dot plot for marker expression
    sc.pl.dotplot(adata, available_markers[:20], 'leiden_1', dendrogram=True,
                  save='_marker_expression.pdf')
    
    print(f"Number of clusters identified: {len(adata.obs['leiden_1'].unique())}")
    print(f"Cluster distribution: {adata.obs['leiden_1'].value_counts().sort_index()}")
    
else:
    print("No AnnData object available for clustering")


In [ ]:
# Extract protein x cell expression matrix
if adata is not None:
    print("Extracting protein × cell expression matrix...")
    
    # Get expression matrix (cells × proteins)
    expression_matrix = adata.X
    protein_names = adata.var_names.tolist()
    cell_ids = adata.obs_names.tolist()
    
    print(f"Expression matrix shape: {expression_matrix.shape}")
    print(f"Number of cells: {len(cell_ids)}")
    print(f"Number of proteins: {len(protein_names)}")
    
    # Create DataFrame for easier handling
    expression_df = pd.DataFrame(
        expression_matrix,
        index=cell_ids,
        columns=protein_names
    )
    
    # Add cell metadata
    cell_metadata = adata.obs.copy()
    cell_metadata['cell_id'] = cell_ids
    
    # Extract spatial coordinates
    spatial_coords = cell_metadata[['x', 'y', 'cell_id', 'unique_region', 'condition']].copy()
    
    # Add cluster information if available
    if 'leiden_1' in cell_metadata.columns:
        spatial_coords['cluster'] = cell_metadata['leiden_1']
        expression_df['cluster'] = cell_metadata['leiden_1']
    
    print(f"\\nSpatial coordinates extracted for {len(spatial_coords)} cells")
    print(f"Coordinate columns: {spatial_coords.columns.tolist()}")
    
    # Display summary statistics
    print("\\nExpression matrix summary:")
    print(expression_df.describe())
    
    print("\\nSpatial coordinates summary:")
    print(spatial_coords.describe())
    
else:
    print("No clustered data available for extraction")
    expression_df = None
    spatial_coords = None
    cell_metadata = None


In [ ]:
# Save expression matrix and spatial coordinates
if expression_df is not None and spatial_coords is not None:
    print("\\nSaving expression matrix and spatial coordinates...")
    
    # Save expression matrix
    expression_file = output_dir / 'protein_x_cell_expression_matrix.csv'
    expression_df.to_csv(expression_file)
    print(f"✓ Expression matrix saved to: {expression_file}")
    
    # Save spatial coordinates
    spatial_file = output_dir / 'cell_spatial_coordinates.csv'
    spatial_coords.to_csv(spatial_file, index=False)
    print(f"✓ Spatial coordinates saved to: {spatial_file}")
    
    # Save complete cell metadata
    metadata_file = output_dir / 'cell_metadata.csv'
    cell_metadata.to_csv(metadata_file)
    print(f"✓ Cell metadata saved to: {metadata_file}")
    
    # Create a summary report
    summary_file = output_dir / 'analysis_summary.txt'
    with open(summary_file, 'w') as f:
        f.write("SPACEc CODEX Analysis Summary\\n")
        f.write("=" * 50 + "\\n\\n")
        f.write(f"Analysis Date: {pd.Timestamp.now()}\\n")
        f.write(f"Selected TMA: {selected_tma_name if 'selected_tma_name' in locals() else 'N/A'}\\n")
        f.write(f"Number of TIF files processed: {len(selected_tifs) if 'selected_tifs' in locals() else 'N/A'}\\n")
        f.write(f"Total cells analyzed: {len(expression_df)}\\n")
        f.write(f"Number of proteins: {len(protein_names)}\\n")
        f.write(f"Number of clusters: {len(adata.obs['leiden_1'].unique()) if 'leiden_1' in adata.obs else 'N/A'}\\n")
        f.write(f"Regions: {', '.join(spatial_coords['unique_region'].unique())}\\n")
        f.write(f"Conditions: {', '.join(spatial_coords['condition'].unique())}\\n")
        f.write("\\nFiles generated:\\n")
        f.write(f"- {expression_file.name}\\n")
        f.write(f"- {spatial_file.name}\\n")
        f.write(f"- {metadata_file.name}\\n")
        f.write(f"- preprocessed_adata.h5ad\\n")
        f.write("\\nProtein markers analyzed:\\n")
        for i, protein in enumerate(protein_names):
            f.write(f"{i+1:2d}. {protein}\\n")
    
    print(f"✓ Analysis summary saved to: {summary_file}")
    
    # Display file sizes
    print("\\nGenerated files:")
    for file_path in [expression_file, spatial_file, metadata_file, summary_file]:
        if file_path.exists():
            size_mb = file_path.stat().st_size / (1024 * 1024)
            print(f"  {file_path.name}: {size_mb:.2f} MB")
    
else:
    print("No data available to save")


In [ ]:
# Create comprehensive visualizations
if adata is not None and spatial_coords is not None:
    print("Creating comprehensive visualizations...")
    
    # 1. Spatial distribution of clusters
    print("\\n1. Creating spatial cluster plots...")
    
    # Spatial plot of clusters
    sp.pl.catplot(
        adata,
        color="leiden_1",
        unique_region="condition",
        X='x', Y='y',
        n_columns=2,
        palette='tab20',
        savefig=True,
        output_fname="spatial_clusters",
        output_dir=str(output_dir),
        figsize=15,
        size=5
    )
    
    # 2. Expression heatmap
    print("\\n2. Creating expression heatmaps...")
    
    # Select top markers for heatmap
    top_markers = available_markers[:15] if len(available_markers) >= 15 else available_markers
    
    # Create heatmap
    plt.figure(figsize=(12, 8))
    cluster_means = []
    cluster_labels = []
    
    for cluster in sorted(adata.obs['leiden_1'].unique()):
        cluster_cells = adata[adata.obs['leiden_1'] == cluster]
        cluster_mean = cluster_cells.to_df()[top_markers].mean()
        cluster_means.append(cluster_mean)
        cluster_labels.append(f"Cluster {cluster}")
    
    heatmap_data = pd.DataFrame(cluster_means, index=cluster_labels)
    
    sns.heatmap(heatmap_data, annot=True, cmap='RdBu_r', center=0, fmt='.2f')
    plt.title('Mean Protein Expression by Cluster')
    plt.tight_layout()
    plt.savefig(output_dir / 'cluster_expression_heatmap.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # 3. Individual marker spatial plots
    print("\\n3. Creating individual marker spatial plots...")
    
    # Create spatial plots for key markers
    key_markers = ['CD3', 'CD20', 'CD68', 'panCK', 'Ki67']
    key_markers = [m for m in key_markers if m in adata.var_names]
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.flatten()
    
    for i, marker in enumerate(key_markers[:6]):
        ax = axes[i]
        
        # Get expression values
        expression_values = adata.to_df()[marker]
        
        # Create scatter plot
        scatter = ax.scatter(spatial_coords['x'], spatial_coords['y'], 
                           c=expression_values, cmap='viridis', s=1, alpha=0.6)
        
        ax.set_title(f'{marker} Expression')
        ax.set_xlabel('X coordinate')
        ax.set_ylabel('Y coordinate')
        plt.colorbar(scatter, ax=ax, shrink=0.8)
    
    # Hide unused subplots
    for i in range(len(key_markers), len(axes)):
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.savefig(output_dir / 'marker_spatial_expression.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("\\n✓ All visualizations created successfully!")
    
else:
    print("No data available for visualization")
